In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
df_GSW = pd.read_csv("GSW_player_sl.csv")
df_GSW['Time_limit_in_minutes'] = df_GSW['Time_limit']*1.5
df_GSW.iloc[13,0]=0
df_GSW.iloc[12,0]=0
df_GSW.iloc[6,0]=1
df_GSW.iloc[9,0]=1
print(df_GSW[['selected','Player','Pos','Fantasy_point','Time_limit_in_minutes']])

    selected                 Player Pos  Fantasy_point  Time_limit_in_minutes
0          1         Andre Iguodala   G           3.67                 30.795
1          1         Andrew Wiggins   F           2.48                 48.405
2          1          Chris Chiozza   G           2.04                 29.250
3          1             Damion Lee   G           2.01                 36.000
4          1         Draymond Green   F           4.14                 45.195
5          1         Gary Payton II   G           2.52                 23.850
6          1            Jeff Dowtin   G          -0.59                 33.150
7          1       Jonathan Kuminga   C          -0.24                 29.850
8          1           Jordan Poole   G           2.38                 35.250
9          1  Juan Toscano-Anderson   F           0.19                 27.795
10         1           Kevon Looney   C           1.19                 25.350
11         0            Moses Moody   G          -0.12          

## Baseline Model 1st Stage

Select short-listed players (happens before game) - make sure every position have enough players to play through the game.

In [3]:
df2 = df_GSW.copy()

,selected,Player,Pos,Fantasy_point,Time_limit,Time_limit_in_minutes
0,1,Andre Iguodala,G,3.67,20.53,30.795
1,1,Andrew Wiggins,F,2.48,32.27,48.405
2,1,Chris Chiozza,G,2.04,19.50,29.250
3,1,Damion Lee,G,2.01,24.00,36.000
4,1,Draymond Green,F,4.14,30.13,45.195
5,1,Gary Payton II,G,2.52,15.90,23.850
6,1,Jeff Dowtin,G,-0.59,22.10,33.150
7,1,Jonathan Kuminga,C,-0.24,19.90,29.850
8,1,Jordan Poole,G,2.38,23.50,35.250
9,1,Juan Toscano-Anderson,F,0.19,18.53,27.795


In [4]:
N = df2.shape[0]
F = df2["Fantasy_point"].to_list()

z = []
p = []
for i in range(len(df2)):
    if df2.iloc[i,2] == 'G':
        z.append([1,0,0])
        p.append([df2.iloc[i,4]*1.5,0,0])
    elif df2.iloc[i,2] == 'F':
        z.append([0,1,0])
        p.append([0,df2.iloc[i,4]*1.5,0])        
    else:
        z.append([0,0,1])
        p.append([0,0,df2.iloc[i,4]*1.5])

In [5]:
z = np.array(z)
p = np.array(p)
print(z)
print(p)



[[1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]]
[[30.795  0.     0.   ]
 [ 0.    48.405  0.   ]
 [29.25   0.     0.   ]
 [36.     0.     0.   ]
 [ 0.    45.195  0.   ]
 [23.85   0.     0.   ]
 [33.15   0.     0.   ]
 [ 0.     0.    29.85 ]
 [35.25   0.     0.   ]
 [ 0.    27.795  0.   ]
 [ 0.     0.    25.35 ]
 [31.65   0.     0.   ]
 [ 0.     0.    29.55 ]
 [ 0.    31.245  0.   ]
 [47.85   0.     0.   ]]


In [6]:
m = Model("Baseline Model base")
s = m.addVars(N,vtype = GRB.BINARY, name = 'x')
m.setObjective( quicksum(s[i]*F[i] for i in range(N)), GRB.MAXIMIZE)
m.addConstr(quicksum(z[i,0]*p[i,0]*s[i] for i in range(N)) >= 48*2, 'guard time')
m.addConstr(quicksum(z[i,1]*p[i,1]*s[i]  for i in range(N)) >= 48*2, 'forward time')
m.addConstr(quicksum(z[i,2]*p[i,2]*s[i]  for i in range(N)) >= 48, 'center time')
m.addConstr(quicksum(s[i] for i in range(N)) ==12, '12 people team')


# Solving the model
m.optimize()


Academic license - for non-commercial use only - expires 2022-01-04
Using license file /Users/jiankunwang/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x671cd6e1
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+02]
Found heuristic solution: objective 23.2800000
Presolve removed 4 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 32.58 23.28 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.258000000000e+01, best bound 3.258000000000e+01, gap 0.0000%


In [7]:
s = []
for v in m.getVars():
    s.append(v.x)
df2['selected'] = s

In [8]:
df2

,selected,Player,Pos,Fantasy_point,Time_limit,Time_limit_in_minutes
0,1.0,Andre Iguodala,G,3.67,20.53,30.795
1,1.0,Andrew Wiggins,F,2.48,32.27,48.405
2,1.0,Chris Chiozza,G,2.04,19.50,29.250
3,1.0,Damion Lee,G,2.01,24.00,36.000
4,1.0,Draymond Green,F,4.14,30.13,45.195
5,1.0,Gary Payton II,G,2.52,15.90,23.850
6,0.0,Jeff Dowtin,G,-0.59,22.10,33.150
7,0.0,Jonathan Kuminga,C,-0.24,19.90,29.850
8,1.0,Jordan Poole,G,2.38,23.50,35.250
9,1.0,Juan Toscano-Anderson,F,0.19,18.53,27.795


### Baseline Model 2nd Stage

In [9]:
# model set-up

def model_setup():
    
    m = Model("Baseline Model")
    x = m.addVars(T,vtype = GRB.BINARY, name = 'x')
    m.setObjective( quicksum(x[i,j,h] * F[j] for i in range(I) for j in range(J) for h in range(H)), GRB.MAXIMIZE)
    m.addConstrs( (x[i,j,h] <= z[j][h] for i in range(I) for j in range(J) for h in range(H)), "position")
    m.addConstrs( (quicksum(x[i,j,0] for j in range(J)) == 2 for i in range(I)), "guard")
    m.addConstrs( (quicksum(x[i,j,1] for j in range(J)) == 2 for i in range(I)), "forward")
    m.addConstrs( (quicksum(x[i,j,2] for j in range(J)) == 1 for i in range(I)), "center")
    m.addConstrs( (quicksum(x[i,j,h] for i in range(I) for h in range(H)) - quicksum(p[j][h] for h in range(H)) <= gap for j in range(J)), "physical limitation")
    m.setParam('OutputFlag', False)
    return m

In [10]:
#df = df_GSW.iloc[:,:]

df = df2
df = df[df["selected"] == 1].reset_index(drop=True)

In [12]:
# parameters set-up


F = df["Fantasy_point"].to_list()
print("F:",F,"\n")
# F = [6.4,3.2,1.4,0.6,0,0.1,2.1,0.2,-0.4,0.3,2.1,-1.1]

z = []
p = []
for i in range(len(df)):
    if df.iloc[i,2] == 'G':
        z.append([1,0,0])
        p.append([min(df.iloc[i,4]*1.5,48),0,0])
    elif df.iloc[i,2] == 'F':
        z.append([0,1,0])
        p.append([0,min(df.iloc[i,4]*1.5,48),0])        
    else:
        z.append([0,0,1])
        p.append([0,0,min(df.iloc[i,4]*1.5,48)])

print("z:",z,"\n")
print("p:",p)

I = 48
J = 12
H = 3
gap = 0

temp = []
for i in range(I):
    for j in range(J):
        for h in range(H):
            temp.append((i,j,h))
T = tuplelist(temp)

F: [3.67, 2.48, 2.04, 2.01, 4.14, 2.52, 2.38, 0.19, 1.19, 4.26, 3.4, 4.3] 

z: [[1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0]] 

p: [[30.795, 0, 0], [0, 48, 0], [29.25, 0, 0], [36.0, 0, 0], [0, 45.195, 0], [23.85, 0, 0], [35.25, 0, 0], [0, 27.795, 0], [0, 0, 25.349999999999998], [0, 0, 29.549999999999997], [0, 31.244999999999997, 0], [47.849999999999994, 0, 0]]


In [13]:
# setup and solve the modle
m = model_setup()
m.optimize()

#  Print optimal solutions and optimal value
print("\n Optimal solution:")
i_th = 1
j_th = 1
h_th = 1
pos1 = "Guard:  \t"
pos2 = "Forward:\t"
pos3 = "Center:\t"
for var in m.getVars():
    if var.x > 0 and h_th == 1:
        pos1 = pos1 + "Player No." + str(j_th) + "\t"
    elif var.x > 0 and h_th == 2:
        pos2 = pos2 + "Player No." + str(j_th) + "\t"
    elif var.x > 0 and h_th == 3:
        pos3 = pos3 + "Player No." + str(j_th) + "\t"
    if j_th == 12 and h_th == 3:
        print("Minute",str(i_th),":\n")
        print("\t",pos1,"\n\t",pos2,"\n\t",pos3,"\n")
        pos1 = "Guard:  \t"
        pos2 = "Forward:\t"
        pos3 = "Center:\t"
        j_th = 0
        i_th = i_th + 1
    if h_th == 3:
        h_th = 0
        j_th = j_th + 1
    h_th = h_th + 1

print("\n Optimal Score:")
print('Obj:', m.objVal)


 Optimal solution:
Minute 1 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.9	 

Minute 2 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.10	 

Minute 3 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.9	 

Minute 4 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.10	 

Minute 5 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.2	Player No.5	 
	 Center:	Player No.10	 

Minute 6 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.9	 

Minute 7 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.10	 

Minute 8 :

	 Guard:  	Player No.6	Player No.12	 
	 Forward:	Player No.5	Player No.11	 
	 Center:	Player No.10	 

Minute 9 :

	 Guard:  	Player No.1	Player No.12	 
	 Forward:	Player No.5

In [14]:
print("\n Gap \t Optimal Score \t Compared with the previous model")
for i in range(10):
    gap = (i) * 3
    
    # setup and solve the model
    m1 = model_setup()   
    m1.optimize()

    #  print optimal solutions and compare them    
    print(" ",gap,"\t",m1.objVal,"\t",m1.objVal - m.objVal)



 Gap 	 Optimal Score 	 Compared with the previous model
  0 	 847.5299999999983 	 0.0
  3 	 869.7099999999979 	 22.17999999999961
  6 	 885.1299999999975 	 37.59999999999923
  9 	 900.5499999999976 	 53.0199999999993
  12 	 915.9699999999973 	 68.43999999999903
  15 	 931.3899999999971 	 83.85999999999888
  18 	 945.8899999999968 	 98.35999999999854
  21 	 948.9599999999967 	 101.42999999999847
  24 	 948.9599999999967 	 101.42999999999847
  27 	 948.9599999999967 	 101.42999999999847
